In [8]:
import sys
import os
import pandas as pd
sys.path.append("scripts")

In [9]:

from combineSingleDB import load_and_aggregate_sentiment, merge_with_price
# Paths
news_dir = "/Users/natwat/Desktop/CPSC_Projects/Trader/data/news"
price_path = "/Users/natwat/Desktop/CPSC_Projects/Trader/data/features/TSLA_features.csv"  # Must be minute-level
output_path = "/Users/natwat/Desktop/CPSC_Projects/Trader/data/features/TSLA_features_full.csv"

# Run full pipeline
sentiment_df = load_and_aggregate_sentiment(news_dir, resample_interval="1min")
sentiment_df.head()


,Datetime,Prob_Pos_distilbert,Prob_Neu_distilbert,Prob_Neg_distilbert,Prob_Pos_roberta,Prob_Neu_roberta,Prob_Neg_roberta,Prob_Pos_finbert,Prob_Neu_finbert,Prob_Neg_finbert,Sentiment_Avg,Sentiment_Disagreement
0,2025-06-11 13:06:00-07:00,0.003562,0.0,0.996438,0.401107,0.594891,0.004001,0.000006,0.000139,0.999855,0.134892,0.230556
1,2025-06-11 13:07:00-07:00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2025-06-11 13:08:00-07:00,0.008069,0.0,0.991931,0.405738,0.571251,0.023012,0.005559,0.000097,0.994344,0.139789,0.230322
3,2025-06-11 13:09:00-07:00,0.011589,0.0,0.988411,0.403848,0.584442,0.011710,0.007282,0.020265,0.972453,0.140906,0.227725
4,2025-06-11 13:10:00-07:00,0.006503,0.0,0.993497,0.370414,0.564167,0.065419,0.006304,0.415064,0.578633,0.127740,0.210162


In [10]:

from combineSingleDB import merge_with_price


merge_with_price(sentiment_df, price_path, output_path)

# View output
df = pd.read_csv(output_path)
df.head()

✅ Merged minute-level dataset saved to /Users/natwat/Desktop/CPSC_Projects/Trader/data/features/TSLA_features_full.csv


,Datetime,Close,High,Low,Open,Volume,EMA20,Prob_Pos_distilbert,Prob_Neu_distilbert,Prob_Neg_distilbert,Prob_Pos_roberta,Prob_Neu_roberta,Prob_Neg_roberta,Prob_Pos_finbert,Prob_Neu_finbert,Prob_Neg_finbert,Sentiment_Avg,Sentiment_Disagreement,Return,Label
0,2025-06-11 13:30:00,333.989990,334.477295,332.170013,334.394989,12669435.0,326.554726,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000449,1
1,2025-06-11 13:31:00,334.140015,335.500000,332.880096,334.005005,1197084.0,327.277135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002541,1
2,2025-06-11 13:32:00,334.989105,335.070007,333.119995,334.184998,762829.0,328.011608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.002266,0
3,2025-06-11 13:33:00,334.230011,335.230011,333.730011,334.952515,677551.0,328.603837,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.002962,0
4,2025-06-11 13:34:00,333.239990,334.349915,332.369995,334.220001,874896.0,329.045375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001125,1


In [11]:
import ta

df = pd.read_csv("/Users/natwat/Desktop/CPSC_Projects/Trader/data/features/TSLA_features_full.csv", parse_dates=["Datetime"])

# Add rolling indicators
df["SMA_50"] = ta.trend.sma_indicator(df["Close"], window=50)
df["RSI_14"] = ta.momentum.RSIIndicator(df["Close"], window=14).rsi()
df["MACD"] = ta.trend.macd_diff(df["Close"])
vwap = ta.volume.VolumeWeightedAveragePrice(
    high=df["High"], low=df["Low"], close=df["Close"], volume=df["Volume"]
)
df["VWAP"] = vwap.vwap  



In [12]:
df["Return_1min"] = df["Close"].pct_change()
df["Volume_Change"] = df["Volume"].pct_change()
df["Price_Diff"] = df["Close"] - df["Open"]

In [13]:
# Drop rows with NaN from rolling indicators
df.dropna(inplace=True)

# Define your input features
feature_cols = [
    "Close", "High", "Low", "Open", "Volume", "EMA20", "SMA_50", "RSI_14", "MACD", "VWAP",
    "Price_Diff", "Return_1min", "Volume_Change",
    "Prob_Pos_distilbert", "Prob_Neu_distilbert", "Prob_Neg_distilbert",
    "Prob_Pos_roberta", "Prob_Neu_roberta", "Prob_Neg_roberta",
    "Prob_Pos_finbert", "Prob_Neu_finbert", "Prob_Neg_finbert",
    "Sentiment_Avg", "Sentiment_Disagreement"
]

# Your label (used later for evaluation, not part of input state)
label_col = "Label"

In [14]:
df.to_csv("/Users/natwat/Desktop/CPSC_Projects/Trader/data/final/TSLA_ready.csv", index=False)


In [15]:
df.head()

,Datetime,Close,High,Low,Open,Volume,EMA20,Prob_Pos_distilbert,Prob_Neu_distilbert,Prob_Neg_distilbert,...,Sentiment_Disagreement,Return,Label,SMA_50,RSI_14,MACD,VWAP,Return_1min,Volume_Change,Price_Diff
49,2025-06-11 14:19:00,332.144989,332.640015,332.000000,332.575012,292861.0,332.059552,0.000000,0.0,0.000000,...,0.000000,-0.002966,0,331.992899,50.771082,0.068167,332.382167,-0.001383,0.452431,-0.430023
50,2025-06-11 14:20:00,331.160004,332.140015,330.760010,332.109985,521395.0,331.973881,0.020220,0.0,0.979780,...,0.249143,0.001208,1,331.936299,44.062174,-0.035299,332.332423,-0.002966,0.780350,-0.949982
51,2025-06-11 14:21:00,331.559998,331.799988,330.730011,331.100006,285710.0,331.934463,0.000000,0.0,0.000000,...,0.000000,0.001470,1,331.884699,47.118153,-0.074520,332.325293,0.001208,-0.452028,0.459991
52,2025-06-11 14:22:00,332.047485,332.349792,331.021210,331.568207,329584.0,331.945227,0.029515,0.0,0.970485,...,0.030315,-0.000385,0,331.825867,50.656266,-0.065662,332.334021,0.001470,0.153561,0.479279
53,2025-06-11 14:23:00,331.919586,332.179993,331.720001,332.109985,195124.0,331.942785,0.000000,0.0,0.000000,...,0.000000,0.001751,1,331.779658,49.716427,-0.066258,332.336688,-0.000385,-0.407969,-0.190399
